# Preparación del entorno

# Prueba con SQL

In [17]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [18]:
%%sql
select 'hola mundo' 

,'hola mundo'
0,hola mundo


In [19]:
%%sql
create or replace table competencia_02 as 
select 
    *
from read_csv_auto('D:\Equipo\Documentos\Posgrado\2C23\datasets\competencia_02_crudo.csv')

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

RuntimeError: (duckdb.IOException) IO Error: Could not write file ".tmp\duckdb_temp_storage-0.tmp" (error in WriteFile): Espacio en disco insuficiente.

[SQL: create or replace table competencia_02 as 
select 
    *
from read_csv_auto('D:\Equipo\Documentos\Posgrado\2C23\datasets\competencia_02_crudo.csv')]
(Background on this error at: https://sqlalche.me/e/14/e3q8)
If you need help solving this issue, send us a message: https://ploomber.io/community


## Queries Básicas

In [20]:
%sql select * from competencia_02 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,201901,1,0,1,48,171,8304.31,42993.54,2501.04,...,0.0,8,0.00,-35773.32,2.33,5148,36602.13,54,0,5008.71
1,29184630,201901,1,0,1,57,296,18318.76,42597.26,14986.76,...,0.0,1,0.00,-67414.82,1149.57,3299,28503.06,10,0,1395.87
2,29185587,201901,1,0,1,77,296,20862.84,85481.31,4228.40,...,0.0,1,0.00,0.00,0.00,6790,5170.25,2,0,0.00
3,29185646,201901,1,0,1,58,231,25218.94,165317.78,4474.25,...,0.0,22,69653.59,-59422.00,0.00,7037,67991.90,31,0,2897.31
4,29187499,201901,1,0,1,69,349,5307.54,32374.08,4706.22,...,0.0,8,0.00,-4802.21,0.00,7596,4851.93,3,0,492.66


In [21]:
%%sql
select 
    foto_mes
    , count(*) as cantidad -- cuenta cuantos casos hay en cada foto_mes 
                           -- y lo guarda en un campo llamado cantidad
from competencia_02
group by foto_mes

,foto_mes,cantidad
0,201901,124864
1,201902,125898
2,201903,126536
3,201904,127059
4,201905,127683
5,201906,129181
6,201907,130551
7,201908,132615
8,201909,134267
9,201910,136515


In [22]:
%%sql
CREATE VIEW tabla_filtrada AS
SELECT
    *
FROM
    competencia_02
WHERE
    foto_mes IN ('202101', '202102', '202103', '202104', '202105', '202106', '202107');

RuntimeError: (duckdb.CatalogException) Catalog Error: View with name "tabla_filtrada" already exists!
[SQL: CREATE VIEW tabla_filtrada AS
SELECT
    *
FROM
    competencia_02
WHERE
    foto_mes IN ('202101', '202102', '202103', '202104', '202105', '202106', '202107');]
(Background on this error at: https://sqlalche.me/e/14/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


Ejemplo con 6 lags (Agregar una columna llamada "clase_ternaria". En cada fila,, si arranca en 202101, si el lag más alto es 202107, que sea "CONTINUA; si "foto_mes_lag6" es NaN y los otros tienen dato, que sea "BAJA +6.  Si "foto_mes_lag6" y "foto_mes_lag5" es NaN, entonces "clase_ternaria" será "BAJA+5". Luego si "foto_mes_lag6", "foto_mes_lag5" y   "foto_mes_lag4" es NaN, , entonces "clase_ternaria" será "BAJA+4". Y así sucesivamente

 Si el cliente arranca en 202102, el razonamiento es igual, sólo que en vez de "BAJA+6" es "BAJA +5", y así sucesivamente.

Mismos razonamiento si arranca en 202103 (si "BAJA +5", pasa a ser "BAJA+4", y así sucesivamente)., 202104, y así)

In [23]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes,
    LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag1,
    LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag2,
    LEAD(foto_mes, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag3,
    LEAD(foto_mes, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag4,
    LEAD(foto_mes, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag5,
    LEAD(foto_mes, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag6,
    CASE
        WHEN foto_mes = '202101' AND foto_mes_lag6 = '202107' THEN 'CONTINUA'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NOT NULL THEN 'BAJA+6'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'BAJA+5'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+4'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
    END AS clase_ternaria
FROM
    tabla_filtrada;

,numero_de_cliente,foto_mes,foto_mes_lag1,foto_mes_lag2,foto_mes_lag3,foto_mes_lag4,foto_mes_lag5,foto_mes_lag6,clase_ternaria
0,29188883,202101,202102.0,202103.0,202104.0,202105.0,202106.0,202107.0,CONTINUA
1,29188883,202102,202103.0,202104.0,202105.0,202106.0,202107.0,NaN,NaN
2,29188883,202103,202104.0,202105.0,202106.0,202107.0,NaN,NaN,NaN
3,29188883,202104,202105.0,202106.0,202107.0,NaN,NaN,NaN,NaN
4,29188883,202105,202106.0,202107.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1144235,186418975,202106,202107.0,NaN,NaN,NaN,NaN,NaN,NaN
1144236,186418975,202107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1144237,187076825,202106,202107.0,NaN,NaN,NaN,NaN,NaN,NaN
1144238,187076825,202107,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
%%sql
SELECT
    numero_de_cliente,
    foto_mes,
    LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag1,
    LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag2,
    LEAD(foto_mes, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag3,
    LEAD(foto_mes, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag4,
    LEAD(foto_mes, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag5,
    LEAD(foto_mes, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag6,
    CASE
        WHEN foto_mes = '202101' AND foto_mes_lag6 = '202107' THEN 'CONTINUA'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NOT NULL THEN 'BAJA+6'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'BAJA+5'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+4'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
        
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'BAJA+4'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
        
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 

        
    END AS clase_ternaria
FROM
    tabla_filtrada;

,numero_de_cliente,foto_mes,foto_mes_lag1,foto_mes_lag2,foto_mes_lag3,foto_mes_lag4,foto_mes_lag5,foto_mes_lag6,clase_ternaria
0,29188883,202101,202102.0,202103.0,202104.0,202105.0,202106.0,202107.0,CONTINUA
1,29188883,202102,202103.0,202104.0,202105.0,202106.0,202107.0,NaN,CONTINUA
2,29188883,202103,202104.0,202105.0,202106.0,202107.0,NaN,NaN,CONTINUA
3,29188883,202104,202105.0,202106.0,202107.0,NaN,NaN,NaN,CONTINUA
4,29188883,202105,202106.0,202107.0,NaN,NaN,NaN,NaN,CONTINUA
...,...,...,...,...,...,...,...,...,...
1144235,186418975,202106,202107.0,NaN,NaN,NaN,NaN,NaN,NaN
1144236,186418975,202107,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1144237,187076825,202106,202107.0,NaN,NaN,NaN,NaN,NaN,NaN
1144238,187076825,202107,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
%%sql
CREATE OR REPLACE VIEW tabla_ternaria AS
SELECT
    numero_de_cliente,
    foto_mes,
    LEAD(foto_mes, 1) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag1,
    LEAD(foto_mes, 2) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag2,
    LEAD(foto_mes, 3) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag3,
    LEAD(foto_mes, 4) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag4,
    LEAD(foto_mes, 5) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag5,
    LEAD(foto_mes, 6) OVER (PARTITION BY numero_de_cliente ORDER BY foto_mes) AS foto_mes_lag6,
    CASE
        WHEN foto_mes = '202101' AND foto_mes_lag6 = '202107' THEN 'CONTINUA'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NOT NULL THEN 'BAJA+6'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'BAJA+5'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+4'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202101' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
        
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'BAJA+4'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202102' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
        
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'BAJA+3'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202103' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'BAJA+2'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202104' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NOT NULL THEN 'CONTINUA'
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NOT NULL THEN 'BAJA+1'
        WHEN foto_mes = '202105' AND foto_mes_lag6 IS NULL AND foto_mes_lag5 IS NULL AND foto_mes_lag4 IS NULL AND foto_mes_lag3 IS NULL AND foto_mes_lag2 IS NULL AND foto_mes_lag1 IS NULL AND foto_mes IS NOT NULL THEN 'CONTINUA'
 

        
    END AS clase_ternaria
FROM
    tabla_filtrada;

,Count


Ahora me quedo sólo con el ID, renombro la columna "clase _ternaria" como "clase01", y creo el view

In [32]:
%%sql
SELECT numero_de_cliente, foto_mes, clase_ternaria as clase01
from tabla_ternaria

,numero_de_cliente,foto_mes,clase01
0,29188883,202101,CONTINUA
1,29188883,202102,CONTINUA
2,29188883,202103,CONTINUA
3,29188883,202104,CONTINUA
4,29188883,202105,CONTINUA
...,...,...,...
1144235,186418975,202106,NaN
1144236,186418975,202107,NaN
1144237,187076825,202106,NaN
1144238,187076825,202107,NaN


In [30]:
%%sql
CREATE OR REPLACE VIEW tabla_clase AS
SELECT numero_de_cliente, foto_mes, clase_ternaria as clase01
from tabla_ternaria

,Count


In [31]:
%%sql
select 
    clase01
    , count(*) as cantidad
from tabla_clase
group by clase01

,clase01,cantidad
0,CONTINUA,800207
1,NaN,329102
2,BAJA+3,3112
3,BAJA+1,4078
4,BAJA+4,1851
5,BAJA+2,3825
6,BAJA+5,1163
7,BAJA+6,902


Hago el join con la tabla original para tener la clase nueva

In [41]:
%%sql
SELECT
    c.*,
    t.clase01
FROM
    competencia_02 c
LEFT JOIN
    tabla_clase t
ON
    c.numero_de_cliente = t.numero_de_cliente AND c.foto_mes = t.foto_mes;

MemoryError: cannot allocate memory for array

In [ ]:
%%sql
CREATE OR REPLACE VIEW competencia_02_final AS
SELECT
    c.*,
    t.clase01
FROM
    competencia_02 c
LEFT JOIN
    tabla_clase t
ON
    c.numero_de_cliente = t.numero_de_cliente AND c.foto_mes = t.foto_mes;

Guardar el archivo

In [ ]:
%%sql

# Guardar en CSV
csv_file_path = 'competencia_02.csv'
competencia_02_final.to_csv(csv_file_path, index=False, header=column_names)

# Comprimir CSV
with open(csv_file_path, 'rb') as f_in:
    with gzip.open('output.csv.gz', 'wb') as f_out:
        f_out.writelines(f_in)